In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

# Problem Set 2 Solutions 

_Contents_
* [Classes](#sec_classes)
* [Problem 1](#sec_prob1)
* [Problem 3](#sec_prob3)


## Class Definitions <a class="anchor" id="sec_classes"></a>

In [2]:
class GateBase(object):
  def __init__(self, t_prop_delay=1e-9, initial_state=False):
    self._current_output_state = initial_state
    self._next_output_state = initial_state # Will move to current after tpd.
    self._time_of_last_state_change = 0
    self._tpd = t_prop_delay
    self._input_states = [] # length of list defined by *argv.

  def get_state(self, t):
    self._update_state(t)
    return int(self._current_output_state)

  def set_input(self, t, *argv):
    if len(self._input_states) != len(argv):
      raise ValueError("Lengths of input argv and input states differ.")

    bool_argv = [bool(in_) for in_ in argv] 
    
    # Check if any input states have changed.
    for ii, new_input_i in enumerate(argv):
      if self._input_states[ii] != new_input_i:
        self._time_of_last_state_change = t # reset the timer
        self._input_states[ii] = new_input_i
    
    self._update_next_output_state()

  def _update_next_output_state(self):
    # Perform the logic action of the gate.
    raise NotImplementedError("Derived class should implement this function.")

  def _update_state(self, t):
    if t - self._time_of_last_state_change >= self._tpd:
      self._current_output_state = self._next_output_state
      #self._time_of_last_state_change = t # reset the timer
    

class NAND2Gate(GateBase):
  def __init__(self, t_prop_delay=1e-9, initial_state=True):
    super().__init__(t_prop_delay, initial_state)
    self._input_states = [False, False]
    self._update_next_output_state()
    
  def _update_next_output_state(self):
    # Perform the logic action of the gate.
    a = self._input_states[0]
    b = self._input_states[1]
    self._next_output_state = not ( a and b )

class AND2Gate(GateBase):
  def __init__(self, t_prop_delay=1e-9, initial_state=True):
    super().__init__(t_prop_delay, initial_state)
    self._input_states = [False, False]
    self._update_next_output_state()
    
  def _update_next_output_state(self):
    # Perform the logic action of the gate.
    a = self._input_states[0]
    b = self._input_states[1]
    self._next_output_state = ( a and b )

class OR3Gate(GateBase):
  def __init__(self, t_prop_delay=1e-9, initial_state=True):
    super().__init__(t_prop_delay, initial_state)
    self._input_states = [False, False, False]
    self._update_next_output_state()
    
  def _update_next_output_state(self):
    # Perform the logic action of the gate.
    a = self._input_states[0]
    b = self._input_states[1]
    c = self._input_states[2]
    self._next_output_state = ( a or b or c )

class NOTGate(GateBase):
  def __init__(self, t_prop_delay=1e-9, initial_state=False):
    super().__init__(t_prop_delay, initial_state)
    self._input_states = [False,]
    self._update_next_output_state()

  def _update_next_output_state(self):
    a = self._input_states[0]
    self._next_output_state = not a

## Problem 1 <a class="anchor" id="sec_prob1"></a>


## Problem 3 <a class="anchor" id="sec_prob3"></a>

In [3]:
t1 = 5
t2 = t1 #10
t3 = t1 #15
t4 = t1 #20
t5 = t1 #25

t = np.linspace(0,200,100001)

# D Latch Specific Gates
G0 = NOTGate(t1, initial_state=True)
G1 = AND2Gate(t2, initial_state=False)
G2 = AND2Gate(t3, initial_state=False)
G3 = AND2Gate(t4, initial_state=False)
G4 = OR3Gate(t5, initial_state=False)

Td = 20
D = 1 + 0*t

T = 50
Gate = 1*(t >= 20)

u1o = 0*t
u2o = 0*t
u3o = 0*t
u4o = 0*t
q = 0*t

for ii, t_ in enumerate(t):
  if ii == 0:
    u1o_ = G0.get_state(t_)
    u2o_ = G1.get_state(t_)
    u3o_ = G2.get_state(t_)
    u4o_ = G3.get_state(t_)
    q_ = G4.get_state(t_)
  else:
    u1o_, u2o_, u3o_ = u1o[ii-1], u2o[ii-1], u3o[ii-1]
    u4o_, q_ = u4o[ii-1], q[ii-1] 
  D_, Gate_ = D[ii], Gate[ii]

  G0.set_input(t_, Gate_)
  G1.set_input(t_, q_, u1o_)
  G2.set_input(t_, q_, D_)
  G3.set_input(t_, Gate_, D_)
  G4.set_input(t_, u2o_, u3o_, u4o_)

  u1o[ii] = G0.get_state(t_)
  u2o[ii] = G1.get_state(t_)
  u3o[ii] = G2.get_state(t_)
  u4o[ii] = G3.get_state(t_)
  q[ii] = G4.get_state(t_)


xfcn = lambda x_ : x_*1
fig, ax = plt.subplots(1,1, figsize=(12,6))
ax.plot(xfcn(t), D+6, label='D')
ax.plot(xfcn(t), Gate+5, label='CLK')
ax.plot(xfcn(t), u1o+4, label='U1o')
ax.plot(xfcn(t), u2o+3, label='U2o')
ax.plot(xfcn(t), u3o+2, label='U3o')
ax.plot(xfcn(t), u4o+1, label='U4o')
ax.plot(xfcn(t), q, label='Q')

ax.grid(True)
leg1 = ax.legend(loc='best')

ax.set_title('Earle Latch Testing')
ax.set_xlabel('Time (ns)')
ax.set_ylabel('Logic Level')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Logic Level')